In [ ]:
import torch
from torch.utils import data
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torchvision import models

from util.LoadDataOCD import LoadsegDBcrop_nia_paper
from util.losses import ComboLoss

import numpy as np
import os
import os.path
from PIL import Image
import csv

In [ ]:
"""------------------------ GPU Set ---------------------------"""
gpu_device = '0' # GPU 번호가 1번인 GPU 사용
"""------------------------ model parameters  ---------------------------"""
batch_size = 2
num_epoch = 50 # epoch
model_mode = 'DeepRes101' # DeepRes101 / DeepRes50 / DeepFCN101

"""------------------------ function parameters ---------------------------"""
weight_decay = 0.0005 # weight 감소량
learning_rate= 1e-3
momentum = 0.9
num_classes = 7  # 0:NO building, 1:소형, 2:아파트, 3:공장, 4:중형단독시설, 5:대형시설, 6:contour
power = 0.9

In [ ]:
img_format = '.png'
gt_format= '.png'
input_size = 512 # 학습으로 들어가는 영상 크기
original_size = 1024 # 데이터셋 영상 크기
csv_data = '/Projects/AISpark_Challenge_IRDIS/Data/AIHub/Training/train_buildings.csv'
data_directory = '/Projects/AISpark_Challenge_IRDIS/Data/AIHub/Training'
img_folder_name = 'train_buildings_data' # DATA_DIRECTORY 내 image 가 들어있는 폴더 이름
label_folder_name = 'train_buildings_labeling' # DATA_DIRECTORY 내 GT image 가 들어있는 폴더 이름

save_pred_every= 20 # 해당 iter 마다 한번씩 저장
snapshot_dir = '/Projects/AISpark_Challenge_IRDIS/Data/AIHub/Training/snapshots_nia_building_resnet50_contour_test/' # 저장 경로

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES']= gpu_device
automated_log_path = snapshot_dir + "log_building.txt" # log 저장 이름
INPUT_SIZE_m = [input_size, input_size]
original_size = [original_size, original_size]  # width, height
IMG_MEAN = np.array((128, 128, 128), dtype=np.float32) # 학습 속도를 위해 RGB 128을 영점으로 둔다. [-128~127], Load code에서 128로 나눔 [-1~0.999]

In [ ]:
"""------------------------ Learning Rate Option ---------------------------"""
def lr_poly(base_lr, iter, max_iter, power):
    return base_lr * ((1 - float(iter) / max_iter) ** (power))

def adjust_learning_rate(optimizer, i_iter):
    lr = lr_poly(learning_rate, i_iter, num_epoch, power)
    optimizer.param_groups[0]['lr'] = lr
    if len(optimizer.param_groups) > 1:
        optimizer.param_groups[1]['lr'] = lr * 10

In [ ]:
"""------------------------ Extraction of matching Label  ---------------------------"""

def LabelTranformer(dataName, label_folder_name):
    """
    dataName: image, label, name 의 정보들
    data_name: post, pre 를 제외한 데이터 이름
    """
    label_set = []
    for i in range(dataName['name'].__len__()):

        # Label (GT) image open
        label_building = Image.open(label_folder_name + '/' + dataName['name'][i] + gt_format)

        if dataName["switching"][i] >= 0.5:
            label_building = label_building.transpose(Image.ROTATE_180)

        label = np.zeros((INPUT_SIZE_m[0], INPUT_SIZE_m[1], num_classes), np.float32)
        # label (buildings)
        label_building = label_building.crop(
            (int(dataName["left"][i]), int(dataName["top"][i]), int(dataName["W"][i]), int(dataName["H"][i])))
        label_building = np.asarray(label_building, np.float32)
        for j in range(num_classes):
            idx_i, idx_j = np.where(label_building[:, :] == j)[:2]
            label[idx_i, idx_j, j] = 1.0  # building ID

        label_set.append(label.copy())

    return label_set

In [ ]:
"""------------------------ MAIN  ---------------------------"""
def main():
    cudnn.enabled = True

    """-------------------------- 개발 MODEL ROAD --------------------------"""
    # DeepRes101 / DeepRes50 / DeepFCN101
    if model_mode == 'DeepRes101':
        model = models.segmentation.deeplabv3_resnet101(pretrained=False, num_classes=num_classes)
    elif model_mode == 'DeepRes50':
        model = models.segmentation.deeplabv3_resnet50(pretrained=False, num_classes=num_classes)
    elif model_mode == 'DeepFCN101':
        model = models.segmentation.fcn_resnet101(pretrained=False, num_classes=num_classes)
    else:
        raise Exception("Please select a model")

    model.cuda(0)
    #     model=nn.DataParallel(model)
    model.train()
    # 쉽게 true로 두면 비용(memory 등) 이 더 들지만 성능이 향상됨.
    cudnn.benchmark = True  # cudnn.benchmark = true -- uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms.
    # -- If this is set to false, uses some in-built heuristics that might not always be fastest.

    """-------------------------- FILE SAVE --------------------------"""
    if not os.path.exists(snapshot_dir):
        os.makedirs(snapshot_dir)
    # log 값을 기록
    with open(automated_log_path, "a") as myfile:
        myfile.write("Epoch\t\titer\t\tloss")

    """-------------------------- FILE LOAD --------------------------"""

    name_list = []
    #     f = open(args.data_dir + args.csv_data, 'r')
    f = open(csv_data, 'r')
    names = csv.reader(f)
    for name in names:
        name[0] = name[0] + gt_format  # csv 파일 list 에 확장자가 빠진 이름들의 list 이므로
        name_list.append(name[0])
    f.close()

    image_list_imgs = name_list

    ITER_SIZE = int(image_list_imgs.__len__() / batch_size)  # training dataset 갯수 / batch_size

    trainloader = data.DataLoader(
        LoadsegDBcrop_nia_paper(data_directory,img_folder_name, image_list_imgs, label_folder_name,
                                mean=IMG_MEAN, crop_size=INPUT_SIZE_m, img_size=original_size,
                                scale=False),
        batch_size=batch_size, shuffle=True, pin_memory=True)


    # implement model.optim_parameters(args) to handle different models' lr setting
    optimizer = optim.SGD(model.parameters(),
                          lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    optimizer.zero_grad()

    # bce_loss = torch.nn.BCEWithLogitsLoss()

    loss_function = ComboLoss({'dice': 1, 'focal': 1}, per_image=True).cuda(0)

    """----------------------- TRAINING START ------------------------"""
    for i_iter in range(num_epoch):
        trainloader_iter = iter(trainloader)
        # 파라미터를 학습 하겠다.

        for param in model.parameters():
            param.requires_grad = True

        adjust_learning_rate(optimizer, i_iter)

        for sub_i in range(ITER_SIZE):
            """----------------------- LOAD DATA ------------------------"""
            batch = trainloader_iter.next()
            # image 하나 뽑기 pre or post image
            image_input, datafiles, data_idx = batch  # img.copy(), label_json_info, np.array(size), data_name

            image_input = image_input.transpose(1, 3)
            image_input = image_input.transpose(2, 3)
            image_input = Variable(image_input).cuda(0)
            # pair image 와 label load
            labels = LabelTranformer(datafiles, data_directory + '/' + label_folder_name)
            # label
            labels = torch.tensor(labels)
            labels = labels.transpose(1, 3)
            labels = labels.transpose(2, 3)
            labels = Variable(labels).cuda(0)

            """----------------------- RESULTS ------------------------"""
            pred_comb = model(image_input)

            # pred_comb = torch.squeeze(pred_comb) # 불필요한 1차원 제거
            # labels = labels.unsqueeze(1) # 필요한 1번째 차원 증가

            """----------------------- BACKWARD ------------------------"""
            loss_target = loss_function(pred_comb['out'], labels)
            # loss_target = loss_function(pred_comb, labels)

            # proper normalization
            loss = loss_target
            """ source  loss, backward for differention """
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()  # weight update

            print('sub_i = {0:3d}/{1:3d},   epoch = {2:3d}/{3:3d},   loss = {4:.3f}'.format(sub_i, ITER_SIZE, i_iter,
                                                                                            num_epoch, loss))
            """----------------------- SAVE WEIGHT FILE ------------------------"""

            with open(automated_log_path, "a") as myfile:  # 원래 있던 값에 덮어쓰기
                myfile.write("\n%d\t\t%d\t\t%.3f" % (i_iter, sub_i, loss))

            if sub_i % save_pred_every == 0 and sub_i != 0:
                print('taking snapshot ...')
                torch.save(model.state_dict(),
                           os.path.join(snapshot_dir, 'sn6_resunet50_' + str(sub_i) + '_ep_' + str(i_iter) + '.pth'))

        print('exp = {}'.format(snapshot_dir))


        if i_iter != 0:
            print('taking snapshot ...')
            torch.save(model.state_dict(), os.path.join(snapshot_dir, 'sn6_resunet50_' + str(i_iter) + '.pth'))

In [ ]:
main()